In [3]:
import requests
import numpy as np
import pandas as pd

# ======================================
# CONFIG
# ======================================
API_BASE_URL = "https://api.marketdata.app/v1"
API_KEY = "YWZ4ZWJVLWNrYlQyZGpYUnpLYUtXZjlhY3ptei1zcjNsbk5fMDg5NXdwZz0"
TICKERS = ["AAPL"]
FROM_DATE = "2024-03-05"
TO_DATE   = "2025-11-21"


# ======================================
# Fetch candles from MarketData.app
# ======================================
def get_mdapp_candles(symbol, resolution="D", from_date=None, to_date=None):
    url = f"{API_BASE_URL}/stocks/candles/{resolution}/{symbol}/"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    params = {"format": "json"}

    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date

    r = requests.get(url, headers=headers, params=params)

    if r.status_code != 200:
        print(f"Error: {symbol} → {r.status_code} {r.text}")
        return None

    return r.json()


# ======================================
# Build DataFrame + indicators
# ======================================
def build_df(data_json):
    df = pd.DataFrame({
        "open": data_json["o"],
        "high": data_json["h"],
        "low": data_json["l"],
        "close": data_json["c"],
        "volume": data_json["v"],
        "t": data_json["t"]
    })

    df = df.sort_values("t").reset_index(drop=True)

    # Indicators
    df['avg_volume'] = df['volume'].rolling(20).mean()
    df['adr'] = df['high'] - df['low']
    df['ma10'] = df['close'].rolling(10).mean()
    df['ma20'] = df['close'].rolling(20).mean()
    df['ma50'] = df['close'].rolling(50).mean()

    # VWAP
    tp = (df['high'] + df['low'] + df['close']) / 3
    df['vwap'] = (tp * df['volume']).cumsum() / df['volume'].cumsum()

    # RSI
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['rsi'] = 100 - (100 / (1 + rs))

    df.fillna(0, inplace=True)
    return df


# ======================================
# Convert df → extras_np (12 columns)
# ======================================
def make_extras_np(df):
    arr = np.column_stack([
        df['open'].to_numpy(),
        df['high'].to_numpy(),
        df['low'].to_numpy(),
        df['close'].to_numpy(),
        df['volume'].to_numpy(),
        df['avg_volume'].to_numpy(),
        df['adr'].to_numpy(),
        df['vwap'].to_numpy(),
        df['ma10'].to_numpy(),
        df['ma20'].to_numpy(),
        df['ma50'].to_numpy(),
        df['rsi'].to_numpy()
    ])
    return arr


# ======================================
# Fair Value Gap Detector
# ======================================
def find_fvg(stocks_data):
    """
    stocks_data: list of (symbol, numpy array)
    array columns:
       0=open, 1=high, 2=low, 3=close, 4=volume, 5=avg_volume
    """

    gaps = {}
    
    for symbol, data in stocks_data:

        gaps[symbol] = []
        
        for i in range(len(data) - 2):
            first_h = data[i][1]
            first_l = data[i][2]
            second_v = data[i+1][4]
            second_avg_v = data[i+1][5]
            third_h = data[i+2][1]
            third_l = data[i+2][2]

            # Require 2nd candle volume ≥ 2× avg volume
            if second_avg_v > 0:
                if second_v >= 2 * second_avg_v:
    
                    # Bullish FVG
                    if first_h < third_l:
                        gaps[symbol].append({
                            "index": i+2,        # third candle index
                            "threshold": third_l, # bullish threshold
                            "type": "bullish"
                        })
                        print(gaps)
                        print(f"{symbol} | Candle {i+2} | 🔵 Bullish FVG "
                              f"(vol {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")
    
                    # Bearish FVG
                    elif first_l > third_h:
                        gaps[symbol].append({
                            "index": i+2,        # third candle index
                            "threshold": third_l, # bullish threshold
                            "type": "bullish"
                        })
                        print(gaps)
                        print(f"{symbol} | Candle {i+2} | 🔴 Bearish FVG "
                              f"(vol {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")

    return gaps


# ======================================
# MAIN EXECUTION
# ======================================
all_stocks_list = []

for symbol in TICKERS:
    print(f"\n=== Fetching {symbol} ===")
    raw = get_mdapp_candles(symbol, from_date=FROM_DATE, to_date=TO_DATE)

    if raw is None:
        continue

    df = build_df(raw)
    extras_np = make_extras_np(df)

    # Only OHLCV + avg volume needed for FVG logic
    ohlc_for_fvg = extras_np[:, :6]

    all_stocks_list.append((symbol, extras_np))

# Run FVG detector
positions = find_fvg(all_stocks_list)


import requests
import numpy as np
import pandas as pd
import json

# -----------------------------
# Configuration
# -----------------------------
API_BASE_URL = "https://api.marketdata.app/v1"
API_KEY = "YWZ4ZWJVLWNrYlQyZGpYUnpLYUtXZjlhY3ptei1zcjNsbk5fMDg5NXdwZz0"  # Optional, if you have a token
STOCK_SYMBOL = "AAPL"

# -----------------------------
# Fetch historical daily candles
# -----------------------------
def get_historical_candles(symbol, resolution="D", from_date=None, to_date=None):
    url = f"{API_BASE_URL}/stocks/candles/{resolution}/{symbol}/"
    params = {
        "format": "json"
    }
    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date

    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching historical candles: {response.status_code} {response.text}")
        return None

# -----------------------------
# Process data into NumPy arrays and indicators
# -----------------------------
dat = get_historical_candles(STOCK_SYMBOL, from_date=FROM_DATE, to_date=TO_DATE)


# your MarketData.app JSON
data_json = dat  # <-- replace with your JSON

# Build DataFrame directly from the lists
df = pd.DataFrame({
    "open": data_json["o"],
    "high": data_json["h"],
    "low": data_json["l"],
    "close": data_json["c"],
    "volume": data_json["v"]
})

# Ensure ascending order by timestamp
df['t'] = data_json["t"]
df = df.sort_values('t').reset_index(drop=True)

# -----------------------------
# Calculate indicators
# -----------------------------
df['avg_volume'] = df['volume'].rolling(window=20).mean()
df['adr'] = df['high'] - df['low']
df['ma10'] = df['close'].rolling(window=10).mean()
df['ma20'] = df['close'].rolling(window=20).mean()
df['ma50'] = df['close'].rolling(window=50).mean()

# VWAP
tp = (df['high'] + df['low'] + df['close']) / 3
df['vwap'] = (tp * df['volume']).cumsum() / df['volume'].cumsum()

# RSI (14-day)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['rsi'] = 100 - (100 / (1 + rs))

# Replace NaNs with zeros
df.fillna(0, inplace=True)

# -----------------------------
# Convert to NumPy arrays
# -----------------------------
opens = df['open'].to_numpy()
highs = df['high'].to_numpy()
lows = df['low'].to_numpy()
closes = df['close'].to_numpy()
volumes = df['volume'].to_numpy()
adr = df['adr'].to_numpy()
vwap = df['vwap'].to_numpy()
avg_volume = df['avg_volume'].to_numpy()
ma10 = df['ma10'].to_numpy()
ma20 = df['ma20'].to_numpy()
ma50 = df['ma50'].to_numpy()
rsi = df['rsi'].to_numpy()

# -----------------------------
# Extras array (all indicators + OHLCV)
# -----------------------------
extras_np = np.column_stack([
    opens, highs, lows, closes, volumes,
    avg_volume, adr, vwap, ma10, ma20, ma50, rsi
])

print("Extras array shape:", extras_np.shape)  # (num_days, 12)
print(extras_np)  # first 5 rows

def check_hits(window, fvg_type, threshold):
    """
    Returns the index (relative to window start) where the price touches the FVG threshold.
    Bullish: low <= threshold
    Bearish: high >= threshold
    """
    #print("HIHIHIHIHIHIH", window)
    if fvg_type == "bullish":
        for i in range(len(window)):
            if window[i][2] < threshold:  # low <= threshold
                #print("LOW", window[i][2])
                #print("WINNY", window)
                return i
    elif fvg_type == "bearish":
        for i in range(len(window)):
            if window[i][1] > threshold:  # high >= threshold
                #print("HIGH", window[i][1])
                #print("WINNY", window)
                return i
    return None  # not touched within window


def get_fvg_candle_windows(extras_np, fvg_list, pre=20, post=50):
    """
    extras_np: full OHLCV+indicators array
    fvg_list: list of dicts like [{'index':33,'threshold':174.5,'type':'bullish'}, ...]
    
    Returns:
    List of dicts for each FVG containing:
      - window (np.array)
      - fvg_index
      - type
      - threshold
      - first_touch_index (relative to window start)
    """
    num_candles = extras_np.shape[0]
    results = []

    for fvg in fvg_list:
        idx = fvg["index"]
        fvg_type = fvg["type"]
        threshold = fvg["threshold"]

        start_idx = max(0, idx - pre)
        end_idx = min(num_candles, idx + post + 1)
        window = extras_np[start_idx:end_idx, :]

        print(fvg)
        #print("EEEEEEE",window[11:])
        #print("EEEEEEE",windows[:1][0][20:])
        first_touch = check_hits(window[11:], fvg_type, threshold)

        results.append({
            "window": window,
            "fvg_index": idx,
            "type": fvg_type,
            "threshold": threshold,
            "first_touch_index": first_touch
        })

    return results

symbol = "AAPL"
extras_np = all_stocks_list[[s for s, _ in all_stocks_list].index(symbol)][1]  # full data
fvg_list = positions[symbol]  # [{'index':..,'threshold':..,'type':..}, ...]

fvg_windows = get_fvg_candle_windows(extras_np, fvg_list, pre=10, post=50)

for fvg in fvg_windows:
    print(f"FVG candle: {fvg['fvg_index']} | Type: {fvg['type']} | Threshold: {fvg['threshold']}")
    print(f"First touch at relative candle: {fvg['first_touch_index']}")
    print("Window shape:", fvg['window'].shape)


=== Fetching AAPL ===
{'AAPL': [{'index': 43, 'threshold': np.float64(180.42), 'type': 'bullish'}]}
AAPL | Candle 43 | 🔵 Bullish FVG (vol 163224109 ≥ 2× avg 65748640)
{'AAPL': [{'index': 43, 'threshold': np.float64(180.42), 'type': 'bullish'}, {'index': 69, 'threshold': np.float64(206.9), 'type': 'bullish'}]}
AAPL | Candle 69 | 🔵 Bullish FVG (vol 172373296 ≥ 2× avg 58582899)
{'AAPL': [{'index': 43, 'threshold': np.float64(180.42), 'type': 'bullish'}, {'index': 69, 'threshold': np.float64(206.9), 'type': 'bullish'}, {'index': 70, 'threshold': np.float64(211.6), 'type': 'bullish'}]}
AAPL | Candle 70 | 🔵 Bullish FVG (vol 198134293 ≥ 2× avg 65869932)
{'AAPL': [{'index': 43, 'threshold': np.float64(180.42), 'type': 'bullish'}, {'index': 69, 'threshold': np.float64(206.9), 'type': 'bullish'}, {'index': 70, 'threshold': np.float64(211.6), 'type': 'bullish'}, {'index': 106, 'threshold': np.float64(201.07), 'type': 'bullish'}]}
AAPL | Candle 106 | 🔴 Bearish FVG (vol 119548589 ≥ 2× avg 58274717

In [27]:
import csv

def export_fvg_to_csv(fvg_windows, filename="data.csv"):
    """
    fvg_windows: list of dicts from get_fvg_candle_windows
    Each row in CSV will contain:
      - pre-FVG candles (flattened)
      - post-FVG candles up to first touch (flattened)
      - type, threshold, rebound (0/1), sector (placeholder)
    """
    all_rows = []

    for fvg in fvg_windows:
        window = fvg["window"]
        print("AAA",window)
        fvg_idx = fvg["fvg_index"]
        print("BBB",fvg_idx)
        fvg_type = fvg["type"]
        print("CCC",fvg_type)
        threshold = fvg["threshold"]
        print("DDD",threshold)
        first_touch_idx = fvg["first_touch_index"]
        print("EEE",first_touch_idx)

        # 10 candles before FVG (first 10 in window slice)
        pre_fvg_data = window[:10, :].flatten()  # 10 candles * 12 columns = 120 elements

        # Data from FVG candle until first touch (if first_touch_index exists)
        if first_touch_idx is not None:
            post_fvg_data = window[10:10+first_touch_idx+1, :].flatten()
        else:
            post_fvg_data = np.array([])  # No touch

        # Flatten everything into a single list
        row = list(pre_fvg_data) + list(post_fvg_data)

        # Add metadata placeholders
        row += [fvg_type, str(threshold), str(first_touch_idx), str(0), "sector_placeholder"]

        all_rows.append(row)

    # Write to CSV
    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        for row in all_rows:
            writer.writerow(row)

    print(f"Saved {len(all_rows)} FVG rows to {filename}")


# =====================
# Example usage
# =====================
export_fvg_to_csv(fvg_windows, filename="data.csv")


AAA [[1.65515000e+02 1.67260000e+02 1.64770000e+02 1.65840000e+02
  4.81164430e+07 5.78276900e+07 2.49000000e+00 1.71396288e+02
  1.69699000e+02 1.69885500e+02 0.00000000e+00 4.46043165e+01]
 [1.65350000e+02 1.67050000e+02 1.64920000e+02 1.66900000e+02
  4.95377610e+07 5.75901616e+07 2.13000000e+00 1.71283691e+02
  1.69422000e+02 1.69688000e+02 0.00000000e+00 4.50980392e+01]
 [1.66540000e+02 1.69300000e+02 1.66210000e+02 1.69020000e+02
  4.82518350e+07 5.71333310e+07 3.09000000e+00 1.71218361e+02
  1.69546000e+02 1.69653500e+02 0.00000000e+00 5.03408316e+01]
 [1.69525000e+02 1.70610000e+02 1.68151100e+02 1.69890000e+02
  5.05583290e+07 5.66475841e+07 2.45890000e+00 1.71182404e+02
  1.69031000e+02 1.69482500e+02 0.00000000e+00 5.05263158e+01]
 [1.69880000e+02 1.71340000e+02 1.69180000e+02 1.69300000e+02
  4.48383540e+07 5.56058674e+07 2.16000000e+00 1.71159098e+02
  1.68306000e+02 1.69373500e+02 0.00000000e+00 5.14700796e+01]
 [1.73370000e+02 1.76030000e+02 1.73100000e+02 1.73500000e+02

In [26]:
row = [""] * 700  # 700 empty columns
row += ["First touch", "Threshold", "Bounce", "Sector"]  # then your headers

import csv

with open("data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(row)